# Customizing OpenAI Gym Environments and Implementing Reinforcement Learning Agents with Stable Baselines

### Theme: Car Racing

- Constança
- Daniela Osório, 202208679
- Inês Amorim, 202108108

---

## Imports

In [ ]:
%pip install -r requirements.txt

In [13]:
import gymnasium as gym
from gymnasium.wrappers import GrayscaleObservation, ResizeObservation, RecordEpisodeStatistics, RecordVideo
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
import os
import gc
from eval import *
#from custom_cr import CustomCarRacing

---

## 1. Introduction

The CarRacing-v3 environment from Gymnasium (previously Gym) is part of the Box2D environments, and it offers an interesting challenge for training reinforcement learning agents. It's a top-down racing simulation where the track is randomly generated at the start of each episode. The environment offers both continuous and discrete action spaces, making it adaptable to different types of reinforcement learning algorithms.

- **Action Space:**

   - **Continuous:** Three actions: steering, gas, and braking. Steering ranges from -1 (full left) to +1 (full right).
   -  **Discrete:** Five possible actions: do nothing, steer left, steer right, gas, and brake.

- **Observation Space:**

    - The environment provides a 96x96 RGB image of the car and the track, which serves as the state input for the agent.

- **Rewards:**

    - The agent receives a -0.1 penalty for every frame, encouraging efficiency.
    - It earns a positive reward for visiting track tiles: the formula is Reward=1000−0.1×framesReward=1000−0.1×frames, where "frames" is the number of frames taken to complete the lap. The reward for completing a lap depends on how many track tiles are visited.

- Episode Termination:

    - The episode ends either when all track tiles are visited or if the car goes off the track, which incurs a significant penalty (-100 reward).

In [2]:
env = gym.make("CarRacing-v3", continuous=False, render_mode='rgb_array') 
obs, info = env.reset()
#continuous = False to use Discrete space

In [9]:
#check render modes
print(env.metadata["render_modes"])

['human', 'rgb_array', 'state_pixels']


- Checking if everything is okay and working

In [10]:
# Reset the environment and render the first frame
obs, info = env.reset()

# Close the environment
env.close()

print("Environment initialized successfully!")

Environment initialized successfully!


In [11]:
print("Action space:", env.action_space)

Action space: Discrete(5)


In [12]:
print("Action Space:", env.action_space)
print("Observation Space:", env.observation_space)
print("Environment Metadata:", env.metadata)


Action Space: Discrete(5)
Observation Space: Box(0, 255, (96, 96, 3), uint8)
Environment Metadata: {'render_modes': ['human', 'rgb_array', 'state_pixels'], 'render_fps': 50}


In [3]:
obs = env.reset()
for _ in range(10):
    """action = env.action_space.sample()  # Random action
    print(f"Action before step: {action}, Type: {type(action)}")
    obs, reward, done, info = env.step(action)"""
    env.step(env.action_space.sample())

env.close()

---
## 2. Training with DQN

Deep Q-Learning (DQN) is a reinforcement learning algorithm that extends the traditional Q-Learning method using neural networks to approximate the Q-values for state-action pairs.
DQN is inherently designed for discrete action spaces, as the neural network outputs a separate Q-value for each action. For each state, the algorithm selects actions based on the highest Q-value, making it ideal for problems where actions are discrete and finite. This is a key advantage compared to other reinforcement learning methods, which may require modifications or different approaches for discrete action selection.

In [6]:
MODELS_DIR = '../../labiacd/models/#isia'

### 2.1. Baseline

In [2]:
env = gym.make("CarRacing-v3", continuous=False)

In [4]:
obs = env.reset()
print(obs[0].shape)

(96, 96, 3)


In [7]:
#create directories
logs_dir = 'DQN_baseline_logs'
logs_path = os.path.join(MODELS_DIR, logs_dir)
os.makedirs(logs_path, exist_ok=True)

video_dir = os.path.join(logs_path, "videos")
tensorboard_dir = os.path.join(logs_path, "tensorboard")
model_dir = os.path.join(logs_path, "models")
os.makedirs(video_dir, exist_ok=True)
os.makedirs(tensorboard_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [ ]:
# Simple DQN architecture
policy_kwargs = dict(net_arch=[64, 64])  # Simpler architecture with 2 layers of 64 units

# Set up the model with simpler hyperparameters
model = DQN('CnnPolicy', env, policy_kwargs=policy_kwargs, 
            verbose=1, buffer_size=50000, learning_starts=1000, 
            learning_rate=0.0005, batch_size=32, exploration_fraction=0.1,
            exploration_final_eps=0.05)

# Setup evaluation and checkpoint callbacks
eval_callback = EvalCallback(env, best_model_save_path=model_dir,
                             log_path=model_dir, eval_freq=5000, n_eval_episodes=5,
                             deterministic=True, render=False)

checkpoint_callback = CheckpointCallback(save_freq=10000, save_path=model_dir,
                                         name_prefix='dqn_model_checkpoint')



# Start training the model with callbacks for evaluation and checkpoints
model.learn(total_timesteps=1_000_000, callback=[eval_callback, checkpoint_callback])

# Save the final model after training
model.save("dqn_car_racing_model")
env.close()

In [ ]:
env.close()
del env
foo = gc.collect()

---

In [26]:
env = gym.make("CarRacing-v3", continuous=False)

In [27]:
obs = env.reset()
print(obs[0].shape)

(96, 96, 3)


In [28]:
#create directories
logs_dir = 'PPO_baseline_logs'
logs_path = os.path.join(MODELS_DIR, logs_dir)
os.makedirs(logs_path, exist_ok=True)

video_dir = os.path.join(logs_path, "videos")
tensorboard_dir = os.path.join(logs_path, "tensorboard")
model_dir = os.path.join(logs_path, "models")
os.makedirs(video_dir, exist_ok=True)
os.makedirs(tensorboard_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [29]:
env = ResizeObservation(env, shape=(64,64))
env = GrayscaleObservation(env, keep_dim=True)

In [30]:
#env = RecordVideo(env, video_folder=video_dir, episode_trigger= lambda t: t % 100000 == 0, disable_logger=True)

In [31]:
obs = env.reset()

In [ ]:
# Simple DQN architecture
policy_kwargs = dict(net_arch=[256])  # Simpler architecture with 2 layers of 64 units

# Set up the model with simpler hyperparameters
model = PPO('CnnPolicy', env, policy_kwargs=policy_kwargs, 
            verbose=1, learning_rate=0.0005, batch_size=128)

# Setup evaluation and checkpoint callbacks
eval_callback = EvalCallback(env, best_model_save_path=model_dir,
                             log_path=model_dir, eval_freq=10000, n_eval_episodes=5,
                             deterministic=True, render=False)

checkpoint_callback = CheckpointCallback(save_freq=10000, save_path=model_dir,
                                         name_prefix='model_checkpoint')



# Start training the model with callbacks for evaluation and checkpoints
model.learn(total_timesteps=1_000_000, callback=[eval_callback, checkpoint_callback])

# Save the final model after training
model.save("ppo_baseline_model")
env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/home/inesamorim46/3ano1sem/isia/reinforcement-learning-with-gymnasium/.venv/lib/python3.10/site-packages/stable_baselines3/common/callbacks.py:418: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x7bd24c3f0940> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7bd243dc0040>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -49.1    |
| time/              |          |
|    fps             | 44       |
|    iterations      | 1        |
|    time_elapsed    | 46       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -46.1       |
| time/                   |             |
|    fps                  | 38          |
|    iterations           | 2           |
|    time_elapsed         | 105         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011774453 |
|    clip_fraction        | 0.125       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.6        |
|    explained_variance   | 0.0157      |
|    learning_rate        | 0.

/home/inesamorim46/3ano1sem/isia/reinforcement-learning-with-gymnasium/.venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=-30.39 +/- 39.60
Episode length: 317.00 +/- 45.77
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 317         |
|    mean_reward          | -30.4       |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.015941966 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.49       |
|    explained_variance   | 0.27        |
|    learning_rate        | 0.0005      |
|    loss                 | 0.245       |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0111     |
|    value_loss           | 0.628       |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -30.8 

In [ ]:
env.close()
del env
foo = gc.collect()

### 2.2. Environment Modifications

In [ ]:
#create directories
logs_dir = 'DQN_env_mod_logs'
logs_path = os.path.join(MODELS_DIR, logs_dir)
os.makedirs(logs_path, exist_ok=True)

tensorboard_dir = os.path.join(logs_path, "tensorboard")
model_dir = os.path.join(logs_path, "models")
os.makedirs(tensorboard_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

### 2.3. Rewards Modifications

In [ ]:
#create directories
logs_dir = 'DQN_rewards_mod_logs'
logs_path = os.path.join(MODELS_DIR, logs_dir)
os.makedirs(logs_path, exist_ok=True)

tensorboard_dir = os.path.join(logs_path, "tensorboard")
model_dir = os.path.join(logs_path, "models")
os.makedirs(tensorboard_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

### 2.4. Environment and Rewards Modifications

In [ ]:
#create directories
logs_dir = 'DQN_env_rewards_mod_logs'
logs_path = os.path.join(MODELS_DIR, logs_dir)
os.makedirs(logs_path, exist_ok=True)

tensorboard_dir = os.path.join(logs_path, "tensorboard")
model_dir = os.path.join(logs_path, "models")
os.makedirs(tensorboard_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

---

# 3. Evaluation

In [ ]:
# Adjust number of episodes based on the environment's characteristics
if hasattr(env, "max_episode_steps"):
    # If the environment has predefined max steps, use a higher number for evaluation
    num_episodes = 50  
else:
    # For simpler environments, use fewer episodes
    num_episodes = 20

In [ ]:
model = foo #load best model

In [ ]:
obs = env.reset()
episode_rewards = []
for episode in range(num_episodes):
    total_reward = 0
    while True:
        action = model.predict(obs)  # Use trained policy
        obs, reward, done, info = env.step(action)
        total_reward += reward
        record_agent_dynamics(env)  # Record smoothness metrics
        if done:
            break
    episode_rewards.append(total_reward)
    obs = env.reset()

print(f"Average Reward: {np.mean(episode_rewards)}")
